In [8]:
#lang iracket/lang #:require sicp

(define pi 3.14)

In [97]:
; lib
(define (close? u v) (< (abs (- u v)) 0.001))
(define (fixed-point f i-guess)
    (define (try g)
        (if (close? (f g) g)
            (f g)
            (try (f g))))
    (try i-guess))

(define (average a b)
    (/ (+ a b) 2.0))

(define (average-damp f)
    (lambda (x)
        (average x (f x))))
(define (sqrt x)
    (fixed-point
        (average-damp
            (lambda (y) (/ x y)))
        1))

(define (square x) (* x x))

(square (sqrt 4))
(define (println a) (display a) (display "\n"))

1


In [23]:
;;; Project 1, 6.001, Spring 2005

;;; idea is to simulate a baseball robot

;; imagine hitting a ball with an initial velocity of v 
;; at an angle alpha from the horizontal, at a height h
;; we would like to know how far the ball travels.

;; as a first step, we can just model this with simple physics
;; so the equations of motion for the ball have a vertical and a 
;; horizontal component

;; the vertical component is governed by
;; y(t) = v sin alpha t + h - 1/2 g t^2 
;; where g is the gravitational constant of 9.8 m/s^2

;; the horizontal component is governed by
;; x(t) = v cos alpha t
;; assuming it starts at the origin

;; First, we want to know when the ball hits the ground
;; this is governed by the quadratic equation, so we just need to know when 
;; y(t)=0 (i.e. for what t_impact is y(t_impact)= 0).
;; note that there are two solutions, only one makes sense physically

(define square
  (lambda (x) (* x x)))

;; these are constants that will be useful to us
(define gravity 9.8)  ;; in m/s
(define pi 3.14159)

In [15]:
;; Problem 1
; this function compute the position with a:accer v: initial velocity u: initial position t: elpased time
; result = 1/2 * a * t ^2 + v * t + u
(define position
  (lambda (a v u t)
    (+ u 
       (* v t) 
       (* 0.5 a (square t)))))

;; you need to complete this procedure, then show some test cases

; (position 0 0 0 0)
; (position 0 0 20 0)
; (position 0 5 10 10)
; (position 2 2 2 2)
; (position 5 5 5 5)

60

In [73]:
;; Problem 2
(define (inner a b c) (- (square b) (* 4 a c)))
(define root1
    (lambda (a b c)
    (let ((inner-val (inner a b c)))
        (if (< inner-val 0)
            #f
            (/ (+ (- b) (sqrt inner-val))
                (* 2 a)))
    )))

(define root2
    (lambda (a b c)
    (let ((inner-val (inner a b c)))
        (if (< inner-val 0)
            #f
            (/ (- (- b) (sqrt inner-val))
                (* 2 a)))
    )))

;; complete these procedures and show some test case
(define (test-root a b c)
    ; (display (inner a b c))
    ; (display " | ")
    (display (root1 a b c))
    (display " | ")
    (display (root2 a b c)))

; (test-root 5 3 6)
(define ax (* (- 0.5) gravity))
(display (inner ax gravity (- 4.9)))

0.0

In [74]:
;; Problem 3
;; I choose root2 is because the inneer value of b^2 - 4ac is always greater than -b thus the result of root2 > 0
(define time-to-impact
    (lambda (vertical-velocity elevation)
    (root2 ax vertical-velocity elevation)
    ))

;; Note that if we want to know when the ball drops to a particular height r 
;; (for receiver), we have

(define time-to-height
    (lambda (vertical-velocity elevation target-elevation)
        (time-to-impact vertical-velocity (- elevation target-elevation))
    ))
(time-to-height gravity 0 4.9)
; (test-root ax 9.18 (- 4.59))

1.0000996492346939

In [90]:
;; Problem 4

;; once we can solve for t_impact, we can use it to figure out how far the ball went

;; conversion procedure
(define degree2radian
    (lambda (deg)
    (/ (*  deg pi) 180.)))

(define travel-distance-simple
    (lambda (elevation velocity angle)
    (let ((vertical-velocity (* (sin (degree2radian angle)) velocity))
            (horizontal-velocity (* (cos (degree2radian angle)) velocity))
            )
        (define elapsed (time-to-impact vertical-velocity elevation))
        (* elapsed horizontal-velocity)
        )
    ))

(display (travel-distance-simple 0 gravity 45))

;; let's try this out for some example values.  Note that we are going to 
;; do everything in metric units, but for quaint reasons it is easier to think
;; about things in English units, so we will need some conversions.

(define meters-to-feet
    (lambda (m)
    (/ (* m 39.6) 12)))

(define feet-to-meters
    (lambda (f)
    (/ (* f 12) 39.6)))

(define hours-to-seconds
    (lambda (h)
    (* h 3600)))

(define seconds-to-hours
    (lambda (s)
    (/ s 3600)))

;; what is time to impact for a ball hit at a height of 1 meter
;; with a velocity of 45 m/s (which is about 100 miles/hour)
;; at an angle of 0 (straight horizontal)
;; at an angle of (/ pi 2) radians or 90 degrees (straight vertical)
;; at an angle of (/ pi 4) radians or 45 degrees

;; what is the distance traveled in each case?
;; record both in meters and in fee

9.799999999991376

In [108]:

;; Problem 5

;; these sound pretty impressive, but we need to look at it more carefully

;; first, though, suppose we want to find the angle that gives the best
;; distance
;; assume that angle is between 0 and (/ pi 2) radians or between 0 and 90
;; degrees

(define alpha-increment 0.01)

(define find-best-angle
    (lambda (velocity elevation)
    (define (try angle max-dist max-angle)
        (let ((dist (travel-distance-simple elevation velocity angle)))
            (if (< dist max-dist)
                max-angle
                (try (+ alpha-increment angle) dist angle)))
        )
    (try 0 0 0)
        )
    )
(define elevation 0)
(define velocity gravity)
(println (find-best-angle velocity elevation))
(define angle-dist (lambda (angle) 
    (travel-distance-simple elevation velocity angle)
))
(println (angle-dist 45))
(println (angle-dist 30))
(println (angle-dist 30.0000000001918))
(println (angle-dist 30.170000000001916))
;; find best angle
;; try for other velocities
;; try for other heights

44.999999999999616
9.799999999991376
8.487044622913961
8.487044622946769
8.515972227847813


In [ ]:
;; Problem 6

;; problem is that we are not accounting for drag on the ball (or on spin 
;; or other effects, but let's just stick with drag)
;;
;; Newton's equations basically say that ma = F, and here F is really two 
;; forces.  One is the effect of gravity, which is captured by mg.  The
;; second is due to drag, so we really have
;;
;; a = drag/m + gravity
;;
;; drag is captured by 1/2 C rho A vel^2, where
;; C is the drag coefficient (which is about 0.5 for baseball sized spheres)
;; rho is the density of air (which is about 1.25 kg/m^3 at sea level 
;; with moderate humidity, but is about 1.06 in Denver)
;; A is the surface area of the cross section of object, which is pi D^2/4 
;; where D is the diameter of the ball (which is about 0.074m for a baseball)
;; thus drag varies by the square of the velocity, with a scaling factor 
;; that can be computed

;; We would like to again compute distance , but taking into account 
;; drag.
;; Basically we can rework the equations to get four coupled linear 
;; differential equations
;; let u be the x component of velocity, and v be the y component of velocity
;; let x and y denote the two components of position (we are ignoring the 
;; third dimension and are assuming no spin so that a ball travels in a plane)
;; the equations are
;;
;; dx/dt = u
;; dy/dt = v
;; du/dt = -(drag_x/m + g_x)
;; dv/dt = -(drag_y/m + g_y)
;; we have g_x = - and g_y = - gravity
;; to get the components of the drag force, we need some trig.
;; let speeed = (u^2+v^2)^(1/2), then
;; drag_x = - drag * u /speed
;; drag_y = - drag * v /speed
;; where drag = beta speed^2
;; and beta = 1/2 C rho pi D^2/4
;; note that we are taking direction into account here

;; we need the mass of a baseball -- which is about .15 kg.

;; so now we just need to write a procedure that performs a simple integration
;; of these equations -- there are more sophisticated methods but a simple one 
;; is just to step along by some step size in t and add up the values

;; dx = u dt
;; dy = v dt
;; du = - 1/m speed beta u dt
;; dv = - (1/m speed beta v + g) dt

;; initial conditions
;; u_0 = V cos alpha
;; v_0 = V sin alpha
;; y_0 = h
;; x_0 = 0

;; we want to start with these initial conditions, then take a step of size dt
;; (which could be say 0.1) and compute new values for each of these parameters
;; when y reaches the desired point (<= 0) we stop, and return the distance (x)

(define drag-coeff 0.5)
(define density 1.25)  ; kg/m^3
(define mass .145)  ; kg
(define diameter 0.074)  ; m
(define beta (* .5 drag-coeff density (* 3.14159 .25 (square diameter))))

(define integrate
    (lambda (x0 y0 u0 v0 dt g m beta)
    YOUR-CODE-HERE))

(define travel-distance
    YOUR-CODE-HERE)


;; RUN SOME TEST CASES

;; what about Denver